In [ ]:
import mcstasscript as ms
import mcstasscript.jb_interface as ms_widget
import scipp as sc
import plopp as pp
import numpy as np

import matplotlib.pyplot as plt
#%matplotlib widget

import make_QENS_instrument

In [ ]:
my_configurator = ms.Configurator()
my_configurator.set_mcrun_path("/usr/bin/")
my_configurator.set_mcstas_path("/usr/share/mcstas/3.3/")

# QENS exercise
This notebook contains code and questions for a McStas simulation of a simplified backscattering instrument that can investigate quasi-elastic scattering from samples. Quasi-elastic scattering is inelastic scattering with small transfers and typically views a broadening of the elastic signal. At ESS the backscattering instrument under construction is called MIRACLES and uses an inverse time of flight technique, here neutrons are scattered of the sample and some hit an analyzer afterwards. This analyzer is angled such that the neutron is scattered almost backwards, and due to Braggs law this will happen with a given energy. It turns out the precision of that energy is highest when the neutron is scattered back in the direction it came from, but most instruments choose a slightly lower angle to avoid hitting the sample a second time. The detector is then placed slightly above or below the sample. Since the analyzer choose a specific energy, the final energy of the neutrons being recorded in the detector is known, this can be used to propagate the time of the neutron to the sample position. Then the time at that moment and the known pulse time can be used to calculate the time-of-flight, which with the known distance gives the speed and thus energy before scattering in the sample. The difference between the known initial and final energy provide the energy transfer, which for backscattering can be down to $\mu$eV, where most other inelastic techniques look at meV.

In this notebook you will get this simplified backscattering instrument and answer a few questions about the results. You will also get to improve it and run experiments with a small range of known and unknown samples.

### Get an instrument object
Here from a function, in future could be online repository

In [ ]:
instrument = make_QENS_instrument.make(input_path="run_folder")
nexus_mode = False

## Investigate instrument
First investigate the instrument object *instrument* using some of the available methods. All the methods that help do that start with the word show. In particular, look at what parameters are available and take a look at the instrument geometry.

In [ ]:
instrument.show_parameters()

In [ ]:
instrument.set_parameters(sample_distance=8, energy_width_ueV=3,
                          sample_choice='"Elastic"', n_pulses=1)
instrument.show_instrument()

## Set parameters
Before running the instrument we need to set some parameters. The most important one is the *sample_distance* parameter describing the distance between the source and the sample. Given the need for high precision in determining the energy of the neutron, which of the following instrument lengths should be chosen?

- A: 30 m
- B: 60 m
- C: 150 m

In [ ]:
# quiz C

Set the *sample_distance* corresponding to the answer above and set the simulated energy width to 3 $\mu$eV.

In [ ]:
instrument.set_parameters(sample_distance=150, energy_width_ueV=3, sample_choice='"Elastic"', n_pulses=1)

In [ ]:
# quiz

## Run the simulation
Now the simulation can be executed with the backengine method. Store the returned data in a python variable called data.

In [ ]:
instrument.settings(ncount=1E7, mpi=2, suppress_output=False)
data = instrument.backengine()

In [ ]:
data

In [ ]:
ms.make_sub_plot(data[0:4], figsize=(10, 8), log=False, orders_of_mag=3)

### Questions
Look at the time distribution of the signal, which statement about this data is true?
- A: The data looks like a typical inelastic signal
- B: The data looks like the ESS pulse structure
- C: The data looks like a typical elastic signal
- D: The data looks like the analyzer selected to broad an energy range

In [ ]:
# quiz B

Is this a problem for a backscattering instrument?
- A: Yes, the low time resolution means low energy resolution
- B: No, the low time resolution is not necessary for high energy resolution

In [ ]:
# quiz A 

How can the instrument be improved? 
- A: Add a chopper to control the time aspect
- B: Add a slit before sample to reduce the illuminated area
- C: Add a slit before analyzer to ensure same angle
- D: Add a spin polarizer to select spin state

In [ ]:
# quiz A

## Improve the instrument
In order to improve the performance of the instrument, we will add a McStas component. The first aspect to consider when doing so is where to place it, both in the component sequence and its physical location. We start by looking at the code sequence.

### McStas sequence 
Use either the *show_diagram* or *show_components* method on the instrument object to get an overview of the component sequence in the instrument. Where would you place the new component?

- A: After the source
- B: Before the sample position
- C: After the sample position
- D: After the analyzer

In [ ]:
instrument.show_diagram()

In [ ]:
# quiz A and B correct

### Which component
Now we need to select what type of component to add to the instrument, here we will need the *DiskChopper* component. Use the *component_help* method on the instrument to learn more about this component.

In [ ]:
instrument.component_help("DiskChopper")

### Chopper calculations
When adding a chopper one need to perform some calculations on delay and frequency. For this exercise, those calculations can be added to the instrument using a function in this folder

In [ ]:
make_QENS_instrument.add_chopper_code(instrument)

To see what variables are used in the instrument, one can use the *show_variables* method like below.

In [ ]:
instrument.show_variables()

### Add chopper component and set parameters
Use the *add_component* method on the instrument to add a chopper. Place it in the component sequence by using either the *before* or *after* keyword argument.

Set the parameters:
 - yheight: 0.05 m
 - radius: 0.7 m
 - nslit: 1.0
 - nu, delay and theta_0: To the variables calculated in the instrument (use quotation marks)

In [ ]:
chopper = instrument.add_component("chopper", "DiskChopper", after="source")
chopper.set_parameters(yheight=0.05, radius=0.7, nu="chopper_frequency",
                       nslit=1.0, delay="chopper_delay", theta_0="chopper_theta")

In [ ]:
# quiz(instrument)

### Placing the component in space
The next physical location of the component need to be specified, which is done using the *set_AT* component. This method takes a list of 3 numbers, corresponding to the *x*, *y* and *z* coordinates of the component. One can also specify in what coordinate system one wants to work, which can be that of any preceeding component. Use the *RELATIVE* keyword to work in the *source* coordinate system. The position of the chopper is needed for calculating phase, so it is available as a variable in the instrument, use this variable to set the position.

In [ ]:
chopper.set_AT("chopper_distance", RELATIVE="source")

In [ ]:
# quiz(isntrument)

### Verify new component
Now that the chopper has been added to the instrument, lets show the component sequence again to verify it was added correctly.

In [ ]:
instrument.show_diagram()

## Run improved instrument
Run the improved instrument with the following parameters:
- sample_distance: 150 m
- energy_width_ueV: 5 ueV
- sample_choice: '"Elastic"'
- frequency_multiplier: 10 (This controls the ratio between chopper and source frequency)

In [ ]:
instrument.show_parameters()

In [ ]:
if nexus_mode:
    instrument.settings(ncount=3E7, mpi=6, suppress_output=True,
                        output_path="QENS_elastic_NeXus_3_pulse", NeXus=nexus_mode)
    instrument.set_parameters(sample_distance=150, energy_width_ueV=5,
                              sample_choice='"Elastic"', n_pulses=3,
                              frequency_multiplier=10)

    data_improved = instrument.backengine()

In [ ]:
instrument.settings(ncount=1E7, mpi=6, suppress_output=True,
                    output_path="QENS_elastic_NeXus_1_pulse", NeXus=nexus_mode)
instrument.set_parameters(sample_distance=150, energy_width_ueV=5,
                          sample_choice='"Elastic"', n_pulses=1,
                          frequency_multiplier=10)

data_improved = instrument.backengine()

In [ ]:
ms.make_sub_plot(data_improved[0:4], figsize=(10, 8), log=False, orders_of_mag=4)
#ms.make_sub_plot(data_improved[1], figsize=(10, 8), log=False, orders_of_mag=4)

### Time resolution
- Q: What is the time resolution of the instrument? (at multiplier=10, FWHM)
- A: 0.235 ms

In [ ]:
#quiz

### Run with known calibration sample
We know run with a known calibration sample, its energy width can be adjusted with the gamma_ueV (HWHM). Run with the following parameters:
- sample_choice: '"Known_quasi-elastic"'
- gamma_ueV: 2 ueV
- energy_width_ueV: 12 ueV

In [ ]:
if nexus_mode:
    instrument.settings(ncount=3E7, mpi=6, suppress_output=True,
                        output_path="QENS_known_quasi_elastic_NeXus_3_pulse", NeXus=nexus_mode)
    instrument.set_parameters(sample_distance=150, energy_width_ueV=12,
                              sample_choice='"Known_quasi-elastic"', gamma_ueV=2,
                              n_pulses=3, frequency_multiplier=10)

    data_known = instrument.backengine()

In [ ]:
instrument.settings(ncount=1E7, mpi=6, suppress_output=True,
                    output_path="QENS_known_quasi_elastic_NeXus_1_pulse", NeXus=nexus_mode)
instrument.set_parameters(sample_distance=150, energy_width_ueV=12,
                          sample_choice='"Known_quasi-elastic"', gamma_ueV=2,
                          n_pulses=1, frequency_multiplier=10)

data_known = instrument.backengine()

In [ ]:
ms.make_sub_plot(data_known[0:4], figsize=(10, 8), log=False, orders_of_mag=4)

- Q: What is the time width when using a known sample with 2 ueV broadening?
- A: x s

In [ ]:
#quiz

### Run with unknnown sample
- sample_choice: '"Unknown_quasi-elastic"'
- energy_width_ueV: 20 ueV

In [ ]:
if nexus_mode:
    instrument.settings(ncount=3E7, mpi=6, suppress_output=True,
                        output_path="QENS_unknown_quasi_elastic_NeXus_3_pulse", NeXus=nexus_mode)

    instrument.set_parameters(sample_distance=150, energy_width_ueV=20,
                              sample_choice='"Unknown_quasi-elastic"',
                              n_pulses=3, frequency_multiplier=10)

    data_unknown = instrument.backengine()

In [ ]:
instrument.settings(ncount=1E7, mpi=6, suppress_output=True,
                    output_path="QENS_unknown_quasi_elastic_NeXus_1_pulse", NeXus=nexus_mode)

instrument.set_parameters(sample_distance=150, energy_width_ueV=20,
                          sample_choice='"Unknown_quasi-elastic"',
                          n_pulses=1, frequency_multiplier=10)

data_unknown = instrument.backengine()

In [ ]:
ms.make_sub_plot(data_unknown[0:4], figsize=(10, 8), log=False, orders_of_mag=4)

## Student exercise would end here

### Load data as events and analyze

In [ ]:
def analyze(data):
    event = ms.name_search("signal_tof_event", data)
    weights = event.get_data_column("p")
    y_values = event.get_data_column("y")
    t_values = event.get_data_column("t")

    n_events = len(y_values)
    positions = np.array([np.zeros(n_events), y_values, np.zeros(n_events)])

    events = sc.DataArray(
            data=sc.array(dims=['events'], unit=sc.units.counts, values=weights), #, variances=weights),
            coords={
                #'x': sc.array(dims=['events'], unit='m', values=x_values),
                'y': sc.array(dims=['events'], unit='m', values=y_values),
                #'z': sc.array(dims=['events'], unit='m', values=z_values),
                'position' : sc.vectors(dims=['events'], values=positions.T, unit='m'),
                't': sc.array(dims=['events'], unit='s', values=t_values),
            })
 
    # Time of flight conversion, skipped, single pulse here
    events.coords["t_c"] = events.coords["t"]

    # Set up distances
    analyzer_distance = event.metadata.parameters["analyzer_distance"] 
    detector_distance = event.metadata.parameters["analyzer_distance"]
    
    # Known speed at detector, 
    # RAD2DEG*0.5*atan(detector_offset/analyzer_distance)
    # 2*analyzer_d*sin(DEG2RAD*0.5*(180-2.0*analyzer_angle)))
    selected_wavelength = 6.264583
    selected_wavevector = 2*np.pi/selected_wavelength
    
    # Calculate arrival time at sample
    V2K = 0.001588254 # from m/s to Å^-1
    speed = selected_wavevector / V2K
    time_from_sample_to_detector = (analyzer_distance + detector_distance - 0.03) / speed
    
    events.coords["sample_time"] = events.coords["t"] - sc.scalar(time_from_sample_to_detector, unit="s")
    
    # Instrument focuses at center of pulse, 2.86 ms / 2
    events.coords["sample_time"] = events.coords["sample_time"] - sc.scalar(0.5*2.86E-3, unit="s")
    
    # Calculate energy after sample
    neutron_mass = sc.scalar(1.675E-27, unit="kg")
    elementary_charge = sc.scalar(1.602E-22, unit="J/meV")
    speed = sc.scalar(speed, unit="m/s")
    
    events.coords["final_energy"] = 0.5*neutron_mass*speed**2/elementary_charge
    
    # Calculate energy before sample
    sample_distance = sc.scalar(event.metadata.parameters["sample_distance"], unit="m")
    events.coords["initial_speed"] = sample_distance/events.coords["sample_time"]
    events.coords["initial_energy"] = 0.5*neutron_mass*events.coords["initial_speed"]**2/elementary_charge
    
    # Calculate energy transfer
    events.coords["energy_transfer"] = events.coords["initial_energy"] - events.coords["final_energy"]
        
    return events

def remove_common_fields(dicts):
    if not dicts:
        return []

    # Get all keys
    all_keys = set(dicts[0].keys())
    
    for d in dicts[1:]:
        all_keys &= set(d.keys())

    # Find keys that have the same value in all dictionaries
    common_keys = {k for k in all_keys if all(d[k] == dicts[0][k] for d in dicts)}

    # Remove these keys from each dictionary
    reduced_dicts = [{k: v for k, v in d.items() if k not in common_keys} for d in dicts]

    return reduced_dicts




In [ ]:
data_dict = dict(instrument=data, improved_instrument=data_improved,
                 known_sample=data_known, unknown_sample=data_unknown)

data_dict = dict(improved_instrument=data_improved,
                 known_sample=data_known, unknown_sample=data_unknown)

plot_dict = {}
for key in data_dict:
    events = analyze(data_dict[key])
    plot_dict[key] = events.bin(energy_transfer=200).hist()
    
pp.plot(plot_dict)

### Results
- HWHM on improved instrument: 5 ueV
- HWHM on known sample: 7 ueV (known width: 2 ueV)
- Unknown mix of 2: 2 3.5 ueV and 0.8 ueV